# 2. Debugging forward-reverse conjugate gradient

* There is some error between these two processes
* Don't know if it is from convergence of CG, but error seems to converge
* Do not know if this effects learning at all. This error could accumulate...

In [9]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

import numpy as np
import torch
import torch.nn
import matplotlib.pyplot as plt
import meld.recon as recon
from meld.util import getAbs, getPhase
# local
import mri
import hqs
import model
import dataloader
import lib_complex as cp

# Setup device
device_no = 1
torch.cuda.set_device(device_no)
device = torch.device("cuda:"+str(device_no) if torch.cuda.is_available() else "cpu")

np_dtype = np.float32

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
Np = [5,5]
Nbatch = 2
A = np.random.randn(*Np).astype(np_dtype) # matrix
b = np.random.randn(Nbatch, Np[0], 1).astype(np_dtype) # input

At = torch.from_numpy(A).to(device)
bt = torch.from_numpy(b).to(device)

ATbt = torch.matmul(At.t(),bt)
ATAt = torch.matmul(At.t(),At)

# Q = lambda x : torch.matmul(ATAt[None,...],x)

def Q(x):
    out = torch.zeros(x.shape[0], ATAt.shape[0], 1).to(device)
    for ii in range(Nbatch):
        out[ii,...] = torch.matmul(ATAt,x[ii,...])
    return out

In [11]:
for nn in range(15):
    output = mri.conjgrad(ATbt*0, ATbt, Q, max_iter = nn, eps = 1e-9)
    reverse = Q(output)
    mse = torch.mean((ATbt - reverse)**2).cpu().numpy()
    print('MSE({1:d}):{0:.8f}'.format(mse, nn))

MSE(0):7.58503437
MSE(1):0.95059550
MSE(2):0.24050450
MSE(3):0.02397619
MSE(4):0.00281426
MSE(5):0.00000011
MSE(6):0.00000000
MSE(7):0.00000000
Hit eps after 7
MSE(8):0.00000000
Hit eps after 7
MSE(9):0.00000000
Hit eps after 7
MSE(10):0.00000000
Hit eps after 7
MSE(11):0.00000000
Hit eps after 7
MSE(12):0.00000000
Hit eps after 7
MSE(13):0.00000000
Hit eps after 7
MSE(14):0.00000000
